In [1]:
# If you want to specify the package address
# you can add them to the PYTHONPATH environment variable.
# Also you can add them on the run time uncommenting the lines below
# import sys
# open3SPN2_HOME = '/Users/weilu/open3spn2/'
# openAWSEM_HOME = '/Users/weilu/openmmawsem/'
# sys.path.insert(0,open3SPN2_HOME)
# sys.path.insert(0,openAWSEM_HOME)

In [2]:
#Import openAWSEM, open3SPN2 and other libraries
import open3SPN2
import ffAWSEM

import pandas
import numpy as np
import simtk.openmm

from functools import partial
import sys

In [3]:
#Fix the system (adds missing atoms)
fix=open3SPN2.fixPDB("1lmb.pdb")

In [4]:
#Create a table containing both the proteins and the DNA
complex_table=open3SPN2.pdb2table(fix)

# Create a single memory file
ffAWSEM.create_single_memory(fix)

chain A is a DNA chain. it will be removed
chain B is a DNA chain. it will be removed
C 87
D 92


/home/cab22/anaconda3/envs/open3SPN2dev/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 4071
  warnings.warn(


In [5]:
#Generate a coarse-grained model of the DNA molecules
dna_atoms=open3SPN2.DNA.CoarseGrain(complex_table)

#Generate a coarse-grained model of the Protein molecules
protein_atoms=ffAWSEM.Protein.CoarseGrain(complex_table)

In [6]:
#Merge the models
Coarse=pandas.concat([protein_atoms,dna_atoms],sort=False)
Coarse.index=range(len(Coarse))
Coarse['serial']=list(Coarse.index)

In [7]:
#Save the protein_sequence
ffAWSEM.save_protein_sequence(Coarse,sequence_file='protein.seq')

In [8]:
# Create a merged PDB
ffAWSEM.writePDB(Coarse,'clean.pdb')

In [9]:
#Create the merged system

pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField(ffAWSEM.xml,open3SPN2.xml)
s=forcefield.createSystem(top)

In [10]:
#Create the DNA and Protein Objects
dna=open3SPN2.DNA.fromCoarsePDB('clean.pdb')
with open('protein.seq') as ps:
    protein_sequence_one=ps.readlines()[0]
protein=ffAWSEM.Protein.fromCoarsePDB('clean.pdb',sequence=protein_sequence_one)
dna.periodic=False
protein.periodic=False

In [11]:
#Copy the AWSEM parameter files
ffAWSEM.copy_parameter_files()

In [12]:
#Clear Forces from the system
keepCMMotionRemover=True
j=0
for i, f in enumerate(s.getForces()):
    if keepCMMotionRemover and i == 0 and f.__class__ == simtk.openmm.CMMotionRemover:
        # print('Kept ', f.__class__)
        j += 1
        continue
    else:
        # print('Removed ', f.__class__)
        s.removeForce(j)
if keepCMMotionRemover == False:
    assert len(s.getForces()) == 0, 'Not all the forces were removed'
else:
    assert len(s.getForces()) <= 1, 'Not all the forces were removed'


In [13]:
#Initialize the force dictionary    
forces={}
for i in range(s.getNumForces()):
    force = s.getForce(i)
    force_name="CMMotionRemover"

#Add 3SPN2 forces
for force_name in open3SPN2.forces:
    print(force_name)
    force = open3SPN2.forces[force_name](dna)
    if force_name in ['BasePair','CrossStacking']:
        force.addForce(s)
    else:
        s.addForce(force)
    forces.update({force_name:force})

#Add AWSEM forces
openAWSEMforces = dict(Connectivity=ffAWSEM.functionTerms.basicTerms.con_term,
                       Chain=ffAWSEM.functionTerms.basicTerms.chain_term,
                       Chi=ffAWSEM.functionTerms.basicTerms.chi_term,
                       Excl=ffAWSEM.functionTerms.basicTerms.excl_term,
                       rama=ffAWSEM.functionTerms.basicTerms.rama_term,
                       rama_pro=ffAWSEM.functionTerms.basicTerms.rama_proline_term,
                       contact=ffAWSEM.functionTerms.contactTerms.contact_term,
                       frag  = partial(ffAWSEM.functionTerms.templateTerms.fragment_memory_term, 
                                       frag_file_list_file="./single_frags.mem", 
                                       npy_frag_table="./single_frags.npy", 
                                       UseSavedFragTable=False, 
                                       k_fm=0.04184/3),
                       beta1 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_1,
                       beta2 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_2,
                       beta3 = ffAWSEM.functionTerms.hydrogenBondTerms.beta_term_3,
                       pap1 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_1,
                       pap2 = ffAWSEM.functionTerms.hydrogenBondTerms.pap_term_2,
                      )
protein.setup_virtual_sites(s)

#Add DNA-protein interaction forces
for force_name in open3SPN2.protein_dna_forces:
    print(force_name)
    force = open3SPN2.protein_dna_forces[force_name](dna,protein)
    s.addForce(force)
    forces.update({force_name: force})

#Fix exclussions
for force_name in openAWSEMforces:
    print(force_name)
    if force_name in ['contact']:
        force = openAWSEMforces[force_name](protein, withExclusion=False,periodic=False)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
    elif force_name in ['Excl']:
        force = openAWSEMforces[force_name](protein)
        print(force.getNumExclusions())
        open3SPN2.addNonBondedExclusions(dna,force)
        print(force.getNumExclusions())
    else:
        force = openAWSEMforces[force_name](protein)
    s.addForce(force)
    forces.update({force_name: force})

Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics
ExclusionProteinDNA
ElectrostaticsProteinDNA
Connectivity
Chain
Chi
Excl
1205
1844
rama
rama_pro
contact
Number of atom:  1171 Number of residue:  179
Contact cutoff  1.0 nm
NonbondedMethod:  1
0
639
frag
Loading Fragment files(Gro files)
Saving fragment table as npy file to speed up future calculation.
All gro files information have been stored in the ./single_frags.npy.             
You might want to set the 'UseSavedFragTable'=True to speed up the loading next time.             
But be sure to remove the .npy file if you modify the .mem file. otherwise it will keep using the old frag memeory.
beta1
beta_1 term ON
beta2
beta_2 term ON


/home/cab22/anaconda3/envs/open3SPN2dev/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


beta3
beta_3 term ON
pap1
pap_1 term ON
No ssweight given, assume all zero
pap2
pap_2 term ON
No ssweight given, assume all zero


In [14]:
# Set up the simulation
temperature=300 * simtk.openmm.unit.kelvin
platform_name='OpenCL' #'Reference','CPU','CUDA', 'OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator, platform)
simulation.context.setPositions(coord)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)

-1175.169189453125


In [15]:
#Obtain total energy

energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print('TotalEnergy',round(energy,6),energy_unit.get_symbol())

#Obtain detailed energy

energies = {}
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

TotalEnergy -1175.169189 kJ/mol
Bond 74.775894 kJ/mol
Angle 955.948914 kJ/mol
Stacking 203.565979 kJ/mol
Dihedral -390.606323 kJ/mol
BasePair -284.232208 kJ/mol
CrossStacking -47.586143 kJ/mol
Exclusion 23.991552 kJ/mol
Electrostatics 23.268274 kJ/mol
ExclusionProteinDNA 296.033478 kJ/mol
ElectrostaticsProteinDNA -10.459806 kJ/mol
Connectivity 1899.296875 kJ/mol
Chain 1899.296875 kJ/mol
Chi 1899.296875 kJ/mol
Excl 1899.296875 kJ/mol
rama -1363.522705 kJ/mol
rama_pro -1363.522705 kJ/mol
contact -1041.547729 kJ/mol
frag -1213.29834 kJ/mol
beta1 -300.796692 kJ/mol
beta2 -300.796692 kJ/mol
beta3 -300.796692 kJ/mol
pap1 0.0 kJ/mol
pap2 0.0 kJ/mol


In [16]:
#Add simulation reporters
dcd_reporter=simtk.openmm.app.DCDReporter(f'output.dcd', 1000)
energy_reporter=simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,
                                                   potentialEnergy=True, temperature=True)
simulation.reporters.append(dcd_reporter)
simulation.reporters.append(energy_reporter)

In [17]:
#Run simulation
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(10000)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)"
1000,2.0000000000000013,-3328.59716796875,293.1424558023211
2000,3.999999999999781,-3186.1337890625,313.6496469731764
3000,5.999999999999561,-3327.55517578125,304.99102397155957
4000,7.999999999999341,-3346.600341796875,294.8169070970988
5000,10.000000000000009,-3214.92236328125,312.86381176843753
6000,12.000000000000677,-3311.098876953125,311.88489116307704
7000,14.000000000001345,-3277.739990234375,316.75323589481184
8000,16.00000000000201,-3304.09375,304.5962416819966
9000,18.000000000000902,-3525.001953125,296.006023759356
10000,19.999999999999794,-3444.7939453125,305.587684915532


In [18]:
#Get the detailed energy after the simulation
energies = {}
for force_name, force in forces.items():
    group=force.getForceGroup()
    state = simulation.context.getState(getEnergy=True, groups=2**group)
    energies[force_name] =state.getPotentialEnergy().value_in_unit(energy_unit)

for force_name in forces.keys():
    print(force_name, round(energies[force_name],6),energy_unit.get_symbol())

Bond 64.278122 kJ/mol
Angle 127.990654 kJ/mol
Stacking -393.611511 kJ/mol
Dihedral -462.269592 kJ/mol
BasePair -244.238144 kJ/mol
CrossStacking -52.877998 kJ/mol
Exclusion 0.705345 kJ/mol
Electrostatics 24.311039 kJ/mol
ExclusionProteinDNA -18.493774 kJ/mol
ElectrostaticsProteinDNA -8.917 kJ/mol
Connectivity 1538.048096 kJ/mol
Chain 1538.048096 kJ/mol
Chi 1538.048096 kJ/mol
Excl 1538.048096 kJ/mol
rama -1514.310669 kJ/mol
rama_pro -1514.310669 kJ/mol
contact -1379.609131 kJ/mol
frag -934.499146 kJ/mol
beta1 -173.023819 kJ/mol
beta2 -173.023819 kJ/mol
beta3 -173.023819 kJ/mol
pap1 0.0 kJ/mol
pap2 0.0 kJ/mol
